In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os 
import glob
import sys
import eppy
import pandas as pd
import numpy as np
from eppy.results import readhtml, fasthtml # the eppy module with functions to read the html
import pprint
import eppy_multi
if sys.platform == 'win32':
    module_path = r'C:\Users\Justin\Documents\GitHub\IPV_Workbench'
else:
    module_path = "/Users/jmccarty/Data/221205_ipv_workbench/github/IPV_Workbench"
print(module_path)
sys.path.insert(0, module_path)
from ipv_workbench.utilities import utils, time_utils
from ipv_workbench.solver import calculations as ipv_calc
from ipv_workbench.solver import simple_power_models as ipv_pv
from ipv_workbench.translators import mapping_irradiance as ipv_irrad

C:\Users\Justin\Documents\GitHub\IPV_Workbench


# General Management

In [8]:
if sys.platform == 'win32':
    project_dir = r'C:\Users\Justin\Documents\GitHub\cmip6_and_buildings'
else:
    project_dir = "/Users/jmccarty/Github/cmip6_an_buildings"
    
vintages = ['pre1980', 'post1980', 'new']
map_vintages = dict(zip(vintages,[0,1,2]))

cz_list = ['1a','2a','2b','3a','3b','3bc','3c','4a','4b',
           '4c','5a','5b','6a','6b','7a','8a']
map_cz = dict(zip(cz_list,np.arange(0,len(cz_list))))

scen_years = eppy_multi.build_scen_years()
scen_years = [f'{a}_{b}' for a,b in eppy_multi.build_scen_years()]
map_scen_years = dict(zip(scen_years,np.arange(0,len(scen_years))))


pv_data_sheet = os.path.join(module_path, 'ipv_workbench', 'devices', 'default_devices', 'cell_module_datasheet.csv')
cell_module = pd.read_csv(pv_data_sheet)
module_area = cell_module.loc[0]['module_height'] * cell_module.loc[0]['module_width'] / 1e6
peak_power = cell_module.loc[0]['Wp']
gamma = cell_module.loc[0]['gamma_ref']

bldg_types = ['RefBldgFullServiceRestaurant',
                'RefBldgHospital',
                'RefBldgLargeHotel',
                'RefBldgLargeOffice',
                'RefBldgMediumOffice',
                'RefBldgMidriseApartment',
                'RefBldgOutPatient',
                'RefBldgPrimarySchool',
                'RefBldgQuickServiceRestaurant',
                'RefBldgSecondarySchool',
                'RefBldgSmallHotel',
                'RefBldgSmallOffice',
                'RefBldgStand-aloneRetail',
                'RefBldgStripMall',
                'RefBldgSuperMarket',
                'RefBldgWarehouse']
map_bldg_type = dict(zip(bldg_types,np.arange(0,len(bldg_types))))

def reverse_dict(my_dict):
    return dict(zip(my_dict.values(),my_dict.keys()))

all_dict = {'vintage':map_vintages,
            'vintage_r':reverse_dict(map_vintages),
            'cz':map_cz,
            'cz_r':reverse_dict(map_cz),
            'scen_year':map_scen_years,
            'scen_year_r':reverse_dict(map_scen_years),
            'bldg_type':map_bldg_type,
            'bldg_type_r':reverse_dict(map_bldg_type)}

map_perf_metrics = {'hs_el_kwh':13,
                    'hs_ng_kwh':14,
                    'cs_el_kwh':15,
                    'fan_el_kwh':16,
                    'fac_ng_kwh':17,
                    'fac_el_kwh':18,
                    'geff_irrad_kwh':19, 
                    'cell_temp_degc':20, 
                    'pv_yield_kwh':21,
                    'fac_el_net_kwh':22,
                    'pv_consumed_kwh':23, 
                    'pv_excess_kwh':24,
                    'self_suff_pct':25,
                    'self_consume_pct':26, 
                    'op_em_kgco2':27, 
                    'op_em_pv_kgco2':28, 
                    'mit_pot_kgco2':29}

tables_of_interest = ['Discomfort-weighted Exceedance OccupiedHours', 
                      'Comfort and Setpoint Not Met Summary',
                      'Heat Index OccupiedHours',
                      'Unmet Degree-Hours']

annual_comfort_metrics = {
    'Discomfort-weighted Exceedance OccupiedHours':{
	'Very-cold Exceedance OccupantHours [hr]':0,
	'Cool Exceedance OccupantHours [hr]':1,
	'Warm Exceedance OccupantHours [hr]':2,
	'Very-hot Exceedance OccupantHours [hr]':3},
'Comfort and Setpoint Not Met Summary':{
	'Time Setpoint Not Met During Occupied Heating':4,
	'Time Setpoint Not Met During Occupied Cooling':5},
'Heat Index OccupiedHours':{
	'Safe (≤ 26.7°C) [hr]':6,
	'Caution (> 26.7°C, ≤ 32.2°C) [hr]':7,
	'Extreme Caution (> 32.2°C, ≤ 39.4°C) [hr]':8,
	'Danger (> 39.4°C, ≤ 51.7°C) [hr]':9,
	'Extreme Danger (> 51.7°C) [hr]':10},
'Unmet Degree-Hours':{
	'Cooling Setpoint Unmet Occupied Degree-Hours [°C·hr]':11,
	'Heating Setpoint Unmet Occupied Degree-Hours [°C·hr]':12}
}

# load up the emissions data for the USA
us_emissions = glob.glob(os.path.join(r"C:\Users\Justin\Documents\GitHub\grid_carbon_data\open_grid\2019_carbon_accounting_hourly_metric_units","*.csv"))
ba_data = pd.read_excel(r"C:\Users\Justin\Documents\GitHub\grid_carbon_data\open_grid\EIA930_Reference_Tables.xlsx")

us_list = []

for filepath in us_emissions:
    us_list.append(pd.read_csv(filepath)['consumed_co2e_rate_kg_per_mwh_for_electricity'].rename(filepath.split(os.sep)[-1].split(".")[0]))
    
us_grid_data = pd.concat(us_list,axis=1)

mean_emissions = []
for cz in cz_list:
    if cz=='8a':
        cz='7a'
        cz_name = '8a'
    else:
        cz_name = cz
    ba_codes = ba_data[ba_data['Climate Zone']==cz]['BA Code'].tolist()
    get_cols = [c for c in us_grid_data.columns if c in ba_codes]
    mean_emissions.append(us_grid_data[get_cols].mean(axis=1).rename(cz_name))
    
cz_emissions = pd.concat(mean_emissions,axis=1) #gC02e/kWh
cz_emissions = cz_emissions / 1000 #kgCO2e/kWh

# natural gas co2 factor
# https://www.ipcc.ch/site/assets/uploads/2018/03/srccs_annex1-1.pdf
ng_factor = 50 #gCO2/MJ
ng_factor = ng_factor/0.2777 #0.277 kWh per MJ (g/kWh)
ng_factor = ng_factor / 1000 #kgCO2e/kWh
ng_factor

0.18005041411595246

In [42]:

def get_weather_file(project_dir, scen_year, cz):
    if 'historical' in scen_year:
        tmy_file = glob.glob(os.path.join(project_dir, 'weather_files', 'tmy3', f'{cz}*'))[0]
    else:
        scenario = scen_year.split('_')[0]
        year = scen_year.split('_')[1]
        year_bands = eppy_multi.get_year_bands(year)
        tmy_file = os.path.join(project_dir, 'weather_files', 'morphed', cz, scenario, 'EPWs', f'{scenario}_50_{year_bands}.epw')
    return tmy_file

def get_result_path(project_dir, vintage, cz, scen_year, bldg_type, result_request):
    main_result_dir = os.path.join(project_dir, 'eplus_simulations', vintage, 'results')
    if result_request=='eplus_tables':
        bldg_dir = glob.glob(os.path.join(main_result_dir, 'v22_2', cz, scen_year, f'{bldg_type}*'))[0]
        result_file = glob.glob(os.path.join(bldg_dir, '*.htm*'))[0]
    
    elif result_request=='eplus_timeseries':
        bldg_dir = glob.glob(os.path.join(main_result_dir, 'v22_2', cz, scen_year, f'{bldg_type}*'))[0]
        result_files = glob.glob(os.path.join(bldg_dir, '*.csv*'))
        for f in result_files:
            if ('-meter' in f) or ('-ssz' in f) or ('-zsz' in f):
                print(result_files)
            else:
                result_file = f
    elif result_request=='irradiance':
        bldg_str = os.path.join(main_result_dir, 'pv_results', cz, scen_year, f'{bldg_type}*')
        bldg_dir = glob.glob(bldg_str)[0]
        result_dir = os.path.join(bldg_dir, 'irrad','annual_irradiance')
        grid_path = glob.glob(os.path.join(result_dir, 'model', 'grid','*.pts'))[0]
        
        direct_ill_path = glob.glob(os.path.join(result_dir, 'results', 'direct','*.ill'))[0]
        direct_sun_path = glob.glob(os.path.join(result_dir, 'results', 'direct','sun*.txt'))[0]
        
        total_ill_path = glob.glob(os.path.join(result_dir, 'results', 'total','*.ill'))[0]
        total_sun_path = glob.glob(os.path.join(result_dir, 'results', 'total','sun*.txt'))[0]
        
        result_file = (grid_path, direct_ill_path, direct_sun_path, total_ill_path, total_sun_path)
    else:
        print('result_request must be specified as eplus_tables, eplus_timeseries, or irradiance')
        result_file = KeyError
    return result_file
        
def compile_sim_code(all_dict, vintage, cz, scen_year, bldg_type):
    return f"{all_dict['vintage'][vintage]}_{all_dict['cz'][cz]}_{all_dict['scen_year'][scen_year]}_{all_dict['bldg_type'][bldg_type]}"

def reverse_sim_code(all_dict, sim_code):
    sim_el = sim_code.split("_")
    sim_el = [int(e) for e in sim_el]
    vintage = all_dict['vintage_r'][sim_el[0]]
    cz = all_dict['cz_r'][sim_el[1]]
    
    scen_year = all_dict['scen_year_r'][0].replace("_","-")
    scenario = scen_year.split("-")[0]
    year = scen_year.split("-")[1]
    bldg_type = all_dict['bldg_type_r'][sim_el[2]]
    
    return f"{vintage}_{cz}_{scen_year}_{scenario}_{year}_{bldg_type}"

def get_annual_eplus_results(project_dir, vintage, cz, scen_year, bldg_type, table_name):
    fpath = get_result_path(project_dir, vintage, cz, scen_year, bldg_type, 'eplus_tables')
    filehandle = open(fpath, 'r') # get a file handle to the html file
    named_table = fasthtml.tablebyname(filehandle, table_name)
    filehandle.close()
    return pd.DataFrame(named_table[1][1:],columns=named_table[1][0]).set_index('',drop=True)

def calc_pv_yield(project_dir, vintage, cz, scen_year, bldg_type, module_data):
    weather_file = get_weather_file(project_dir, scen_year, cz)
    tmy_df = utils.tmy_to_dataframe(weather_file)
    tmy_location = utils.tmy_location(weather_file)
    
    module_area = module_data.loc[0]['module_height'] * module_data.loc[0]['module_width'] / 1e6
    peak_power = module_data.loc[0]['Wp']
    gamma = module_data.loc[0]['gamma_ref']
    
    irrad_results = get_result_path(project_dir, vintage, cz, scen_year, bldg_type, 'irradiance')
    direct_ill = utils.build_full_ill(irrad_results[2], irrad_results[1]).fillna(0).to_numpy(dtype='float16')
    total_ill = utils.build_full_ill(irrad_results[4], irrad_results[3]).fillna(0).to_numpy(dtype='float16')
    diffuse_ill =  np.where(total_ill < direct_ill, direct_ill * 0.01, total_ill - direct_ill)
    
    grid_data = pd.read_csv(irrad_results[0], sep=' ', header=None, 
                            dtype='float64', names=["X", "Y", "Z",
                                                    "X_v", "Y_v", "Z_v"])
    v1_x = grid_data['X_v'].round(6).unique()[0]
    v2_x = grid_data['X_v'].round(6).unique()[1]
    v1_index = grid_data[grid_data['X_v'].round(6)==v1_x].index
    v2_index = grid_data[grid_data['X_v'].round(6)==v2_x].index
    v1 = tuple(grid_data.loc[v1_index][['X','Y','Z']].iloc[0].to_list())
    v2 = tuple(grid_data.loc[v2_index][['X','Y','Z']].iloc[0].to_list())
    
    g_eff_v1 = ipv_irrad.calculate_effective_irradiance_timeseries(direct_ill[:,v1_index],
                                                    diffuse_ill[:,v1_index],
                                                    v1, 
                                                    np.arange(0,8760),
                                                    tmy_location, 
                                                    tmy_df['atmos_Pa'], 
                                                    tmy_df['drybulb_C'], 
                                                    front_cover_color="clear")

    g_eff_v2 = ipv_irrad.calculate_effective_irradiance_timeseries(direct_ill[:,v2_index],
                                                        diffuse_ill[:,v2_index],
                                                        v2, 
                                                        np.arange(0,8760),
                                                        tmy_location, 
                                                        tmy_df['atmos_Pa'], 
                                                        tmy_df['drybulb_C'], 
                                                        front_cover_color="clear")

    g_eff_wh_m2 = np.hstack([g_eff_v1, g_eff_v2])
    g_eff_kwh = g_eff_wh_m2 * module_area / 1000
    g_eff_kwh_array_ts = g_eff_kwh.sum(axis=1)
    cell_temp = ipv_calc.calculate_cell_temperature(g_eff_kwh_array_ts,
                                                    tmy_df['drybulb_C'],
                                                    )

    pv_yield_kwh_ts = np.vectorize(ipv_pv.pv_watts_method)(g_eff_kwh_array_ts, cell_temp, peak_power, gamma, T_ref=25, G_ref=1000, I_misc=0.1)
    
    return g_eff_kwh_array_ts, cell_temp, pv_yield_kwh_ts

def convert_joules(j):
    return j * 2.77778e-7

def create_hourly_results_df(project_dir, eplus_cols, eplus_rename, cell_module, ng_factor, cz_emissions, v, cz, sy, b):
    eplus_results = get_result_path(project_dir, v, cz, sy, b, 'eplus_timeseries')
    res_df = pd.read_csv(eplus_results, index_col='Date/Time', parse_dates=True)
    if 'Heating:NaturalGas [J](Hourly)' in res_df.columns:
        pass
    else:
        res_df['Heating:NaturalGas [J](Hourly)'] = 0
    res_df.set_index(utils.ts_8760(),inplace=True)
    res_df = convert_joules(res_df[eplus_cols]).rename(columns=dict(zip(eplus_cols, eplus_rename)))
    if b=='RefBldgMidriseApartment' or b=='RefBldgMediumOffice':
        geff_irrad_kwh, cell_temp_degc, pv_yield_kwh = calc_pv_yield(project_dir, v, cz, sy, b, cell_module)
    else:
        geff_irrad_kwh, cell_temp_degc, pv_yield_kwh = (np.zeros((8760,)),np.zeros((8760,)),np.zeros((8760,)))

    res_df['geff_irrad_kwh'] = geff_irrad_kwh
    if type(cell_temp_degc) is np.ndarray:
        res_df['cell_temp_degc'] = cell_temp_degc
    else:
        res_df['cell_temp_degc'] = cell_temp_degc.to_numpy()
    res_df['pv_yield_kwh'] = pv_yield_kwh
    res_df['fac_el_net_kwh'] = np.clip(res_df['fac_el_kwh'] - res_df['pv_yield_kwh'],0,None)
    res_df['pv_consumed_kwh'] = res_df['fac_el_kwh'] - res_df['fac_el_net_kwh']
    res_df['pv_excess_kwh'] = np.clip(res_df['fac_el_kwh'] - res_df['pv_yield_kwh'],None,0).abs()
    res_df['self_suff_pct'] = np.where(res_df['fac_el_kwh'] == 0, 0, 100 * (res_df['pv_consumed_kwh'] / res_df['fac_el_kwh']))

    res_df['self_consume_pct'] = 100 * np.divide(res_df['pv_consumed_kwh'], 
                                        res_df['pv_yield_kwh'], out=np.zeros_like(res_df['pv_consumed_kwh']), 
                                        where= res_df['pv_yield_kwh'] != 0)
    res_df['op_em_kgco2'] = (res_df['fac_el_kwh'].to_numpy() * cz_emissions[cz].to_numpy()) + (res_df['fac_ng_kwh'] * ng_factor)
    res_df['op_em_pv_kgco2'] = (res_df['fac_el_net_kwh'].to_numpy() * cz_emissions[cz].to_numpy()) + (res_df['fac_ng_kwh'] * ng_factor)
    res_df['mit_pot_kgco2'] = res_df['op_em_kgco2'] - res_df['op_em_pv_kgco2']
    return res_df

def create_season_df(df, df_cols, season, sim_code):
    season_df = df[df['season_int']==season][df_cols].sum()
    season_df['cell_temp_degc'] = df[df['season_int']==season]['cell_temp_degc'].mean()
    season_df['self_suff_pct'] = np.where(season_df['fac_el_kwh'] == 0, 0, 100 * (season_df['pv_consumed_kwh'] / season_df['fac_el_kwh']))

    season_df['self_consume_pct'] = 100 * np.divide(season_df['pv_consumed_kwh'], 
                                        season_df['pv_yield_kwh'], out=np.zeros_like(season_df['pv_consumed_kwh']), 
                                        where= season_df['pv_yield_kwh'] != 0)

    return season_df.rename(sim_code)
    

# PV Results


In [46]:
weather_file = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\weather_files\morphed\1a\ssp245\EPWs\ssp245_50_2020-2050.epw"
tmy_df = utils.tmy_to_dataframe(weather_file)
tmy_location = utils.tmy_location(weather_file)


In [12]:
total_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\new\results\pv_results\1a\ssp245_2035\RefBldgMidriseApartmentNew2004\irrad\annual_irradiance\results\total"
direct_dir = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\new\results\pv_results\1a\ssp245_2035\RefBldgMidriseApartmentNew2004\irrad\annual_irradiance\results\direct"

def get_full_ill(results_dir):
    ill_file = glob.glob(os.path.join(results_dir, '*.ill'))[0]
    sun_file = glob.glob(os.path.join(results_dir, 'sun*.txt'))[0]
    return utils.build_full_ill(sun_file, ill_file)

total_ill = get_full_ill(total_dir).fillna(0).to_numpy(dtype='float16')
direct_ill = get_full_ill(direct_dir).fillna(0).to_numpy(dtype='float16')
diffuse_ill =  np.where(total_ill < direct_ill, direct_ill * 0.01, total_ill - direct_ill)



In [16]:

sensor_file = r"C:\Users\Justin\Documents\GitHub\cmip6_and_buildings\eplus_simulations\new\results\pv_results\1a\ssp245_2035\RefBldgMidriseApartmentNew2004\irrad\annual_irradiance\model\grid\RefBldgMidriseApartment.pts"
grid_file = pd.read_csv(sensor_file, sep=' ', header=None, dtype='float64', names=["X", "Y", "Z", "X_v", "Y_v", "Z_v"])
v1_x = grid_file['X_v'].round(6).unique()[0]
v2_x = grid_file['X_v'].round(6).unique()[1]
v1_index = grid_file[grid_file['X_v'].round(6)==v1_x].index
v2_index = grid_file[grid_file['X_v'].round(6)==v2_x].index
v1 = tuple(grid_file.loc[v1_index][['X','Y','Z']].iloc[0].to_list())
v2 = tuple(grid_file.loc[v2_index][['X','Y','Z']].iloc[0].to_list())

In [65]:
g_eff_v1 = ipv_irrad.calculate_effective_irradiance_timeseries(direct_ill[:,v1_index],
                                                    diffuse_ill[:,v1_index],
                                                    v1, 
                                                    np.arange(0,8760),
                                                    tmy_location, 
                                                    tmy_df['atmos_Pa'], 
                                                    tmy_df['drybulb_C'], 
                                                    front_cover_color="clear")

g_eff_v2 = ipv_irrad.calculate_effective_irradiance_timeseries(direct_ill[:,v2_index],
                                                    diffuse_ill[:,v2_index],
                                                    v2, 
                                                    np.arange(0,8760),
                                                    tmy_location, 
                                                    tmy_df['atmos_Pa'], 
                                                    tmy_df['drybulb_C'], 
                                                    front_cover_color="clear")

g_eff_wh_m2 = np.hstack([g_eff_v1, g_eff_v2])
g_eff_kwh = g_eff_wh_m2 * module_area / 1000
g_eff_kwh_array_ts = g_eff_kwh.sum(axis=1)

cell_temp = ipv_calc.calculate_cell_temperature(g_eff_kwh_array_ts,
                                                tmy_df['drybulb_C'],
                                                )

pv_yield_kwh_ts = np.vectorize(ipv_pv.pv_watts_method)(g_eff_kwh_array_ts, cell_temp, peak_power, gamma, T_ref=25, G_ref=1000, I_misc=0.1)

In [146]:
calc_pv_yield(project_dir, vintages[0], '1a', 'ssp126_2035',
                    'RefBldgMidriseApartment', cell_module)

28.2


# EPlus Results

## compile annual metrics

In [52]:
tables = list(annual_comfort_metrics.keys())
annual_results_dict = dict.fromkeys(tables,[])
a_ = []
b_ = []
c_ = []
d_ = []

for v in vintages:
    for cz in cz_list:
        for sy in scen_years:
            for b in bldg_types: 
                
                sim_code = compile_sim_code(all_dict, v, cz, sy, b)
                
                for table in tables:
                    cols = list(annual_comfort_metrics[table].keys())
                    annual_df = get_annual_eplus_results(project_dir, 
                                                        v, 
                                                        cz,
                                                        sy, 
                                                        b,
                                                        table)
                    if table == 'Discomfort-weighted Exceedance OccupiedHours':
                        # annual_df = annual_df
                        annual_df = annual_df[cols].rename(columns=annual_comfort_metrics[table]).loc['Average'].rename(sim_code)
                        a_.append(annual_df)
                    elif table == 'Comfort and Setpoint Not Met Summary':
                        annual_df = annual_df.transpose()
                        annual_df = annual_df[cols].rename(columns=annual_comfort_metrics[table])
                        annual_df = annual_df.iloc[0].rename(sim_code)
                        b_.append(annual_df)
                    elif table == 'Heat Index OccupiedHours':
                        annual_df = annual_df[cols].rename(columns=annual_comfort_metrics[table]).loc['Average'].rename(sim_code)
                        c_.append(annual_df)
                    elif table == 'Unmet Degree-Hours':
                        annual_df = annual_df[cols].rename(columns=annual_comfort_metrics[table]).loc['Average'].rename(sim_code)
                        d_.append(annual_df)
                    else:
                        print("How did you get there", table)
                        
                
annual_results = pd.concat([pd.concat(a_,axis=1).transpose(), 
                            pd.concat(b_,axis=1).transpose(), 
                            pd.concat(c_,axis=1).transpose(), 
                            pd.concat(d_,axis=1).transpose()],axis=1)

idx = annual_results.index
codes = pd.Series(idx).apply(lambda x: reverse_sim_code(all_dict,x)).str.split("_",expand=True)
codes = codes.rename(columns=dict(zip([0,1,2,3,4,5],
                                      ['vintage','cz','scen_year','scenario','year','bldg_type'])))
codes = codes.set_index(idx)
annual_results = annual_results.join(codes)

annual_results.to_csv(os.path.join('output_data','annual_comfort_data.csv'))

## Hourly Eplus

In [53]:
eplus_cols = ['Heating:Electricity [J](Hourly)',
              'Heating:NaturalGas [J](Hourly)',
              'Cooling:Electricity [J](Hourly)',
              'Fans:Electricity [J](Hourly)',
              'NaturalGas:Facility [J](Hourly)',
              'Electricity:Facility [J](Hourly)']

eplus_rename = ['hs_el_kwh',
                'hs_ng_kwh',
                'cs_el_kwh',
                'fan_el_kwh',
                'fac_ng_kwh',
                'fac_el_kwh']


In [39]:
sim_code

'0_0_0_10'

In [56]:

season_dfs = {1:[],
              2:[],
              3:[],
              4:[]}
hour_annual_list = []


for v in vintages:
    for cz in cz_list:
        for sy in scen_years[0:1]:
            for b in bldg_types:
                sim_code = compile_sim_code(all_dict, v, cz, sy, b)
                hour_df = create_hourly_results_df(project_dir, eplus_cols, eplus_rename, cell_module, 
                                   ng_factor, cz_emissions,v, cz, sy, b)
                hour_df_cols = hour_df.columns
                # hour_df = hour_df.rename(columns=map_perf_metrics).round(4)
                hour_df.to_csv(os.path.join('output_data','hourly',f'{sim_code}.csv'))
                for season in [1,2,3,4]:   
                    hour_df['season_int'] = hour_df.index.month%12 // 3 + 1

                    season_labels = {1:'Winter',
                                    2:'Spring',
                                    3:'Summer',
                                    4:'Fall'}
                    hour_df['season'] = hour_df['season_int'].map(season_labels)  
                    
                    season_df = create_season_df(hour_df, hour_df_cols, season, sim_code)
                    season_dfs[season].append(season_df)

                hour_annual = hour_df[hour_df_cols].sum().rename(sim_code)
                hour_annual['cell_temp_degc'] = hour_df['cell_temp_degc'].mean()
                hour_annual['self_suff_pct'] = np.where(hour_annual['fac_el_kwh'] == 0, 0, 100 * (hour_annual['pv_consumed_kwh'] / hour_annual['fac_el_kwh']))

                hour_annual['self_consume_pct'] = 100 * np.divide(hour_annual['pv_consumed_kwh'], 
                                                    hour_annual['pv_yield_kwh'], out=np.zeros_like(hour_annual['pv_consumed_kwh']), 
                                                    where= hour_annual['pv_yield_kwh'] != 0)
                hour_annual_list.append(hour_annual)

for season in [1,2,3,4]: 
    season_df = pd.concat(season_dfs[season],axis=1).transpose().rename(columns=map_perf_metrics).round(2)
    season_df.to_csv(os.path.join('output_data','seasonal',f'{season_labels[season]}.csv'))

hour_annual_all = pd.concat(hour_annual_list,axis=1).transpose().rename(columns=map_perf_metrics).round(2)
comfort_df = pd.read_csv(os.path.join('output_data','annual_comfort_data.csv'),index_col='Unnamed: 0')
annual_results_data = pd.concat([comfort_df, hour_annual_all],axis=1)
annual_results_data.to_csv(os.path.join('output_data','all_annual_data.csv'))

UnboundLocalError: cannot access local variable 'result_file' where it is not associated with a value

In [58]:
reverse_sim_code(all_dict,sim_code)

'post1980_2a_historical-2020_historical_2020_RefBldgFullServiceRestaurant'